In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import os
#### INSTRUCTIONS FOR I/O (PLEASE READ) #######
# Input data files are available in the read-only "../input/" (relative) or '/kaggle/input'(absolute) directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
input_path = '2024-flame-ai-challenge/dataset/'
output_path = 'working/'

In [2]:
train_df = pd.read_csv(os.path.join(input_path,'train.csv'))
test_df = pd.read_csv(os.path.join(input_path,'test.csv'))

In [71]:
#gets test set input
def getTestX(idx):
    csv_file = test_df.reset_index().to_dict(orient='list')
    dir_path = os.path.join(input_path, "test")
    id = csv_file['id'][idx]
    nt, Nx, Ny = csv_file['Nt'][idx], csv_file['Nx'][idx], csv_file['Ny'][idx]
    theta = np.fromfile(os.path.join(dir_path, csv_file['theta_filename'][idx]), dtype="<f4").reshape(nt, Nx, Ny)
    xi_f = np.fromfile(os.path.join(dir_path, csv_file['xi_filename'][idx]), dtype="<f4").reshape(nt, Nx, Ny)
    uin  = np.array(csv_file['u'][idx])
    alpha = np.array(csv_file['alpha'][idx])
    uin = np.full_like(theta, uin)
    alpha = np.full_like(theta, alpha)

    X = np.stack([theta, xi_f, uin, alpha], axis = -1) # (t, Nx, Ny, c) 
    X = torch.tensor(X)
    return id, X

#gets train set input
def getTrainData(idx):
    csv_file = train_df.reset_index().to_dict(orient = 'list')
    dir_path = os.path.join(input_path, "train")
    
    id = csv_file['id'][idx]
    nt, Nx, Ny = csv_file['Nt'][idx], csv_file['Nx'][idx], csv_file['Ny'][idx]
    
    theta = np.fromfile(os.path.join(dir_path, csv_file['theta_filename'][idx]), dtype = "<f4").reshape(nt, Nx, Ny)
    xi_f = np.fromfile(os.path.join(dir_path, csv_file['xi_filename'][idx]), dtype = "<f4").reshape(nt, Nx, Ny)
    
    uin  = np.array(csv_file['u'][idx])
    alpha = np.array(csv_file['alpha'][idx])
    uin = np.full_like(theta, uin)
    alpha = np.full_like(theta, alpha)

    X = np.stack([theta[:-19], xi_f[:-19], uin[:-19], alpha[:-19]], axis = -1) #(t, Nx, Ny, c), t range: 0->t-20
    X = torch.tensor(X)
    
    Y = xi_f 
    Y = torch.tensor(Y)
    return id, X, Y
    
#predicts with input
def predict(idx, model):
    id, X = getTestX(idx)
    X = X.unsqueeze(0)
    y_pred = model(X)
    return id, y_pred

#generates submission with model predictions already in SI units
def generate_submission(model):
    y_preds = {}
    ids = []
    for idx in range(len(test_df)):
        id, y_pred = predict(idx, model) 
        #WARNING tmp should be in SI units
        y_preds[id]= np.array(y_pred).flatten(order='C').astype(np.float32)
        ids.append(id)
    df = pd.DataFrame.from_dict(y_preds,orient='index')
    df['id'] = ids

    #move id to first column
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    #reset index
    df = df.reset_index(drop = True)
    return df

In [78]:
#create a torch model based on linear interpolation of fire spread
# REPLACE THIS WITH YOUR MODEL LOADER TO MAKE YOUR PREDICTIONS

class FireSpreadModel_LSSVM(nn.Module):
    def __init__(self, input_size = 20, output_size = 9, n_predictions = 20):
        super(FireSpreadModel_LSSVM, self).__init__()
        # Linear layer: input_size -> output_size
        # constants
        self.n_predictions = n_predictions
        # 
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, data):
        """
        This model takes in:
        Input: data, packed tensor at certain time step t with dimension (nx, ny, 4)
        
        Outputs:
        - fire_predictions: (n_predictions, nx, ny) 
        """
        # unpack the 
        thetas, fires, u10, slope = self.unpack_data(data)
        
        # init the predicted fire location 
        fire_predictions = torch.zeros((self.n_predictions, fires.shape[0], fires.shape[1]))
        
        # init the current state of fire predictions
        fire_predictions[0, :, :] = fires
        
        for t in range(1, self.n_predictions):
            
            # get the previous fire locations, assuming len(fire_loc) > 0
            # using previous step to predict the next time step
#             if t == 1:
#                 fire_locations = self.get_fire_locations(fires)
#             else:
#                 fire_locations = self.get_fire_locations(fire_predictions[t - 1, :, :].squeeze())
            fire_locations = self.get_fire_locations(fire_predictions[t - 1, :, :].squeeze())
            for f in fire_locations:
                
                # get the fire_neighbors need to be updated 
                fire_neighbors = self.get_neighbors(f[0], f[1], fires.shape[0], fires.shape[1])
                
                # prep the input data for the model 
                input_data = self.prep_input_data(thetas,
                                                  fire_predictions[t - 1, :, :].squeeze(), 
                                                  fire_neighbors, 
                                                  u10,
                                                  slope)
                # return 
                new_fire_status = self.fc(input_data)
                
                # update the predicted fire status in fire_predictions
                self.update_fire_predictions(t, fire_predictions, fire_neighbors, new_fire_status)
                
        
        return fire_predictions
    
    def update_fire_predictions(self, t, fire_predictions, fire_neighbors, new_fire_status):
        """update the fire_predictions 
        """
        for idx in range(len(fire_neighbors)):
            i, j = fire_neighbors[idx]
            if i == -1 and j == -1:
                continue
            fire_predictions[t, i, j] = max(fire_predictions[t, i, j], new_fire_status[idx])
    
    def prep_input_data(self, thetas, fires, fire_neighbors, u10, slope):
        """ wrapping all the data required together to feed into the classifier, returning a tensor
        theta: (nx * ny) tensor
        fires: (nx * ny) tensor
        fire_locations: list of tuples
        u10: single value 
        slope: single value
        """
        # get the neighbor status 
            
        f_status = self.neighbor_to_fire_status(fires, fire_neighbors)
        t_status = self.neighbor_to_fire_status(thetas, fire_neighbors)
        
        # concat all the features, 9 + 9 + 1 + 1 = 20 feature in total 
        input_data = f_status + t_status + [u10.item()] + [slope.item()]
        #
        input_data = torch.tensor(input_data)
        return input_data
    
    def unpack_data(self, data):
        """ data:(nx, ny, 4) tensor containing the input data
            index from 0 -> 3: theta, xi_f, uin, alpha
        """
        # unpacking the data, u and slope are scalar values
        thetas = data[:, :, 0].squeeze()       # thetas, nx * ny
        fires  = data[:, :, 1].squeeze()       # fires, nx * ny
        u10   = data[:, :, 2].squeeze()        # u10, nx * ny
        slope = data[:, :, 3].squeeze()        # slope, nx * ny
        
        u10   = u10.mean()                    # u, only interested in the u as single value
        slope = slope.mean()                  # slope, only interested in the slope as single value
        return thetas, fires, u10, slope
    
    def neighbor_to_fire_status(self, fires, fire_neighbors):
        """get the fire status based on the 
        """
        f_status = []
        for i, j in fire_neighbors:
            if i == -1 and j == -1:
                f_status.append(-1)
            else:
                f_status.append(fires[i, j].item())
        return f_status
    
    def get_fire_locations(self, fires):
        """fires has the dimensions of nx * ny, same as the squeezed fire_predictions
        return a list of tuples 
        """
        fire_locations = []
        for i in range(fires.shape[0]):
            for j in range(fires.shape[1]):
                if fires[i][j] == 1:
                    fire_locations.append((i, j))
        return fire_locations
                
    def get_neighbors(self, x, y, M, N):
        """Returns the list of neighbor coordinates neighbors: 
        * * *     7 8 9
        * o * ==> 4 5 6
        * * *     1 2 3
        """
        neighbors = []
        for i in [-1, 0, 1]:
            for j in [-1, 0, 1]:
                new_x, new_y = x + i, y + j
                if 0 <= new_x < M and 0 <= new_y < N:
                    neighbors.append((new_x, new_y))
                  # if at the boundary, insert -1
                else:  
                    neighbors.append((-1, -1))
                    
        return neighbors

# LSSVM loss: least squares + regularization term
def lssvm_loss_SF(y_pred, y_true, model, c = 1.0):
    """LSSVM Loss, Least-Squares Objective + Regularization
        In this loss model, single final step is used to calc the loss function
    """
    """
    LSSVM loss function: Least squares loss + regularization term
    - y_pred: predicted output from the model, (20, nx, ny)
    - y_true: true target labels, (20, nx, ny)
    - model: the LSSVM model to apply regularization
    - c: regularization constant
    """
 
    # Least Squares loss (MSE)
    least_squares_loss = 0.5 * torch.mean((y_pred[-1, :, :] - y_true[-1, :, :]) ** 2)
    
    # Regularization term: ||w||^2
    regularization_loss = 0.5 * c * torch.sum(model.fc.weight ** 2)
    
    return least_squares_loss + regularization_loss

# LSSVM loss: least squares + regularization term
def lssvm_loss_full(y_pred, y_true, model, c = 1.0):
    """LSSVM Loss, Least-Squares Objective + Regularization
        In this loss model, every single step is included in the loss calculation, not just final step
    """
    """
    LSSVM loss function: Least squares loss + regularization term
    - y_pred: predicted output from the model, (20, nx, ny)
    - y_true: true target labels, (20, nx, ny)
    - model: the LSSVM model to apply regularization
    - c: regularization constant
    """
 
    for t in range(1, y_pred.shape[0]):
        # Least Squares loss (MSE)
        least_squares_loss = 0.5 * torch.mean((y_pred[t, :, :] - y_true[t, :, :]) ** 2)
    
    # Regularization term: ||w||^2
    regularization_loss = 0.5 * c * torch.sum(model.fc.weight ** 2)
    
    return least_squares_loss / (y_pred.shape[0] - 1) + regularization_loss


def train_lssvm(model, X, Y, epochs = 10, lr = 0.01, c = 1.0):
    """ X: (t, nx, ny, 4) tensor, t = 130, time step ranges from 0 - 131
        Y: (t, nx, ny) tensor, t = 130, time step ranges from     1 - 150
        at each time step t, using X to predict Y at time t + 19, 
        then calculate the loss function at t + 19 with Y at the same time 
        (based on the forward function in the sample.)
        Hence, we need Y at t+ 1 to Y at t + 19 to train the model and validate the predictions. 
        
        For each epoch, train through all the time steps, 
    """
    optimizer = optim.Adam(model.parameters(), lr = lr)

    for epoch in range(epochs):
        
        # loop over all the time steps for 1 epoch
        for t in range(X.shape[0]):
            
            model.train()
            optimizer.zero_grad()
        
            # Forward pass, input_data at time step t (nx, ny, 4)
            input_data = X[t, :, :, :].squeeze()
            # output a tensor from time t to t + 19   (20, nx, ny)
            output_data = model(input_data)
            
            # Compute Loss
            
            # Option 1: only final time step is considered
            # loss = lssvm_loss(output_data, Y[t : t + 20, :, :], model, c = c)
            
            # Option 2: 
            loss = lssvm_loss_full(output_data, Y[t : t + 20, :, :], model, c = c)
        
            # Backward pass
            loss.backward()
        
            optimizer.step()
            if t % 70 == 0:
                print(f'T {t}/{X.shape[0]}, Loss: {loss.item()}')
        
        #if epoch % 10 == 0:
        print(f'Epoch {epoch}/{epochs}, Loss: {loss.item()}')
        print("++ == ++ == ")    

In [79]:
"""Full training based on all the training data 
"""
def full_training(model, train_df, epochs = 2, c = 1):
    
    lrs = [0.001]
    for lr in lrs:
        for idx in range(len(train_df)):
            print(f"idx = {idx}")
            id, X, Y = getTrainData(idx)
            train_lssvm(model, X, Y, epochs = 2, lr = lr, c = c)

In [80]:
"""Training section
"""
# read training dataframe
train_df = pd.read_csv(os.path.join(input_path,'train.csv'))

# the mdoel takes input features 20, output feature 9
# input features include 9 fire status, 9 theta status, 1 wind speed, 1 slope 
# output features include 9 updated fire status
input_size = 20
output_size = 9
# Create LSSVM model
model = FireSpreadModel_LSSVM(input_size, output_size)

full_training(model, train_df, epochs = 2, c = 1)

idx = 0
T 0/131, Loss: 1.5402206182479858
T 70/131, Loss: 0.9506608247756958
Epoch 0/2, Loss: 0.6170690655708313
++ == ++ == 
T 0/131, Loss: 0.6123374104499817
T 70/131, Loss: 0.36100420355796814
Epoch 1/2, Loss: 0.22397153079509735
++ == ++ == 
idx = 1
T 0/131, Loss: 0.22188982367515564
T 70/131, Loss: 0.0777876004576683
Epoch 0/2, Loss: 0.02961094118654728
++ == ++ == 
T 0/131, Loss: 0.028793761506676674
T 70/131, Loss: 0.008640266954898834
Epoch 1/2, Loss: 0.0031539052724838257
++ == ++ == 
idx = 2
T 0/131, Loss: 0.0028564108069986105
T 70/131, Loss: 0.000940444937441498
Epoch 0/2, Loss: 0.0008369262795895338
++ == ++ == 
T 0/131, Loss: 0.00048032376798801124
T 70/131, Loss: 0.0009388100006617606
Epoch 1/2, Loss: 0.0008369236020371318
++ == ++ == 
idx = 3
T 0/131, Loss: 0.00043665579869411886
T 70/131, Loss: 0.0006258773501031101
Epoch 0/2, Loss: 0.0009169771801680326
++ == ++ == 
T 0/131, Loss: 0.00043665579869411886
T 70/131, Loss: 0.000625873333774507
Epoch 1/2, Loss: 0.000916977

In [75]:
#predicts with input
def predict(idx, model):
    # get test X
    id, X = getTestX(idx)
    # input data: final time step X
    y_pred = model(X[-1, :, :, :].squeeze())
    return id, y_pred

#generates submission with model predictions already in SI units
def generate_submission(model, test_df):
    y_preds = {}
    ids = []
    for idx in range(len(test_df)):
        id, y_pred = predict(idx, model) 
        #WARNING tmp should be in SI units
        y_preds[id]= y_pred.detach().numpy().flatten(order='C').astype(np.float32)
        ids.append(id)
    df = pd.DataFrame.from_dict(y_preds, orient = 'index')
    df['id'] = ids

    # move id to first column
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    #reset index
    df = df.reset_index(drop=True)

    return df

In [81]:
"""Test section, generate submission files
"""
test_df = pd.read_csv(os.path.join(input_path,'test.csv'))
df = generate_submission(model, test_df)
df.to_csv(os.path.join(output_path, 'submission.csv'),index=False)
print('Generating Submission file ... completed' )

Generating Submission file ... completed


In [ ]:
# output model parameters
for param in model.parameters():
    print(param)

In [70]:
print(X.size())

torch.Size([131, 113, 32, 4])


In [ ]:
"""Testing training
X: (t, nx, ny, 4) tensor, t = 130, time step ranges from 0 - 130
Y: (t, nx, ny) tensor, t = 130, time step ranges from    0 - 149 
at each time step t, using X to predict Y at time t + 19, (based on the forward function in the sample.)
hence we need Y at t+ 1 to Y at t + 19 to train the model and validate the predictions 
"""
# testing 
id, X, Y = getTrainData(1)

# model takes in 'data' as the training / testing input 
# 
data = X
target = Y

# the mdoel takes input features 20, output feature 9
# input features include 9 fire status, 9 theta status, 1 wind speed, 1 slope 
# output features include 9 updated fire status
input_size = 20
output_size = 9

# Create LSSVM model
model = FireSpreadModel_LSSVM(input_size, output_size)

# Train the model
train_lssvm(model, X, Y, epochs = 3, lr = 0.005, c = 1)